# Scenario Data Pre-Processing for Model Input

## Description
The following script processes the technology and scenario data to the required fromat and loads it to the danish ship models input folder.
The user can chosse different cost variation in percentage. The precanteg variates the cost values of end 2050.
Further the change in transport supply by ships, the methane and carbon emission factors con be alternated. In order to conduct the changes for scertain scenarios and technologies, the scenario data has to be modified. Cells containing a one will be subject to the variation, zeros not.

## Content
- Script set-up
- Parameter variation
- Time resolution
- Load data
- Scenarios
- Required directories
- Join data
- CH4 factor
- Output
- Scenario implementation
- Time variant data
- Regulations
- Transport demand
- Biofuels

## Script Set-Up

In [544]:
# Import of required packages
import os
import pandas as pd

## Parameter Variation

In [545]:
# One can choose a cost-variation percentage with -1 < r
r = 0.99
R = str(int(r*100))

In [546]:
# Safety first
if r > 1:
    r /= 100

In [547]:
# One can choose a methane emissions variation percentage with -1 < r
em = -0.9995

In [548]:
# One can choose a carbon emissions variation percentage with -1 < r
ec = 0

In [549]:
# One can choose a transport supply variation percentage with -1 < r
ts = 0

## Time Resolution
Sets the time frame of the model with a given resolution.

Resolution: Yearly

In [550]:
# Create a set of time-steps from initial to last year with yearly steps (1 <= dt)
dt = 1
init_year = 2016
last_year = 2051

year = list(range(init_year, last_year + 1, dt))

# Look at year set
print(year)

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051]


## Load Data

In [551]:
# Load fuel data from csv
fuel = pd.read_csv('data_tec/fuels/fuel_data.csv', encoding='utf8', index_col=0)

In [552]:
# Load ship data from csv
ship = pd.read_csv('data_tec/ships/ship_data.csv', encoding='utf8', index_col=0)

In [553]:
# Load fuel specification variations between first and last year from csv
fuel_rates = pd.read_csv('data_scn/fuel_rates.csv', encoding='utf8', index_col=[0, 1])

In [554]:
# Load ship specification variations between first and last year from csv
ship_rates = pd.read_csv('data_scn/ship_rates.csv', encoding='utf8', index_col=[0, 1])

## Scenarios

In [555]:
# Make list of all scenarios
scn = list(fuel_rates.columns.unique())

## Required Directories

In [556]:
# Make required directories
path = 'model_input/' + R + '/'
for s in scn:
    if not os.path.isdir(path + str(s)):
        os.makedirs(path + str(s))

## Join Data

In [557]:
# Make a column with fuel type information for joining with the fuel data frame
ship['fuel_type'] = ship.index.str.split('_').str[1]

In [558]:
# Join ship and fuel data frames
ship = ship.join(fuel, on=ship.fuel_type)

In [559]:
# Make a column to combine the w2t and t2p co2 emissions
ship['ef-co2-w2p_g-MJ'] = ship['ef-co2-w2t_g-MJ'] + ship['ef-co2-t2p_g-MJ']

In [560]:
# Make a column to combine the w2t and t2p ch4 emissions
ship['ef-ch4-w2p_g-MJ'] = ship['ef-ch4-w2t_g-MJ'] + ship['ef-ch4-t2p_g-MJ']

## CH4 Factor
The 100 year green hous gas potential of methane compared to carbon dioxide, as used by the IPCC.

Source: https://www.ipcc.ch/publications_and_data/ar4/wg1/en/ch2s2-10-2.html

In [561]:
# Apply the ch4 emissions factor
ef_ch4 = 25
ship['ef-ch4-w2p_g-MJ'] *= ef_ch4

## Output

In [562]:
# Write joined dataframe to csv
for s in scn:
    ship.to_csv(path + s + '/ship_data.csv', encoding='utf8')

## Scenario Implementation

In [563]:
# Sum reference scenario and the scenario variations
for i in fuel_rates.columns[fuel_rates.columns!='rs']:
    fuel_rates[i] = fuel_rates[i] * ((1 + fuel_rates.rs) * r) + fuel_rates.rs    

In [564]:
# Sum reference scenario and the scenario variations
for i in ship_rates.columns[ship_rates.columns!='rs']:
    for j in ship_rates.index:
        if j[0] == 'cs': x = r
        if j[0] == 'ef-co2-w2p': x = ec
        if j[0] == 'ef-ch4-w2p': x = em
        if j[0] == 'ts': x = ts
        ship_rates.loc[j, i] = ship_rates.loc[j, i] * ((1 + ship_rates.loc[j, 'rs']) * x) + ship_rates.loc[j, 'rs']

In [565]:
# Make a data frame template for time-variant parameters with years as indices and ship types as columns
tvdf = pd.DataFrame(columns=ship.index,
                    index=year)

In [566]:
# Create time variant dataframes from fuel_rates
for s in scn:
    for i in fuel_rates.index.levels[0]:
        for y in year:
            tvdf.loc[y] = ship[ship.columns[ship.columns.str.contains(str(i))][0]]
        for j in ship.index:
            f = j.split('_')[1]
            x = fuel_rates[s].unstack().loc[str(i), str(f)]
            # Calculate annual changing rate
            x = (1 + x) ** (1 / (len(year) - 1)) - 1
            if x != 0:
                tvdf[str(j)] *= ((1 + x) ** (tvdf.index - year[0]))
        # Write time variant dataframe to csv
        tvdf.to_csv(path + s + '/{}.csv'.format(i), encoding='utf8')

In [567]:
# Create time variant dataframes from ship_rates
for s in scn:
    for i in ship_rates.index.levels[0]:
        for y in year:
            tvdf.loc[y] = ship[ship.columns[ship.columns.str.contains(str(i))][0]]
        for j in ship.index:
            x = ship_rates[s].unstack().loc[str(i), str(j)]
            # Calculate annual changing rate
            x = (1 + x) ** (1 / (len(year) - 1)) - 1
            if x != 0:
                tvdf[str(j)] *= ((1 + x) ** (tvdf.index - year[0]))
        # Write time variant dataframe to csv
        tvdf.to_csv(path + s + '/{}.csv'.format(i), encoding='utf8')

## Regulations

In [568]:
# Load regulation data from csv
reg = pd.read_csv('data_scn/regulations.csv', encoding='utf8', index_col=0)

In [569]:
# Sum best estimate and the scenario variations
for i in reg.columns[reg.columns!='rs']:
    reg[i] += reg.rs

In [570]:
# Write regulation data as csv to model input folder
for s in scn:
    scn_regs = pd.DataFrame(reg[s])
    scn_regs.to_csv(path + s + '/regs.csv', encoding='utf8')

## Transport Demand

In [571]:
# Calculate the initial transport demand for long and short ranges (according to the shipping stock script)
ship['td_total_Ttkm'] = 0.056363616136 + 0.09613698339899997
ship['td_short_Ttkm'] = 0.0200927184
ship['td_noneca_Ttkm'] = 0.09613698339899997

In [572]:
# Make a data frame for the transport demand in each year
tvdf = pd.DataFrame(columns=[['td_total_Ttkm', 'td_short_Ttkm', 'td_noneca_Ttkm']], index=year)

In [573]:
# Write time variant dataframe to csv
for s in scn:
    for i in tvdf.columns:
        r = reg[s].td_change
        for y in year:
            tvdf.loc[y, str(i)] = ship[i].mean()
        # Calculate annual changing rate
        r = (1 + r) ** (1 / (len(year) - 1)) - 1
        if r != 0:
            tvdf[i] *= ((1 + r) ** (tvdf.index - year[0]))
    tvdf.to_csv(path + s + '/td.csv', encoding='utf8')

## Biofuel Availability

In [574]:
# Make a data frame for the biofuel availability in each year
tvdf = pd.DataFrame(columns=['ba_PJ'], index=year)

In [575]:
# Calculate the biofuel availability for first and last year
for s in scn:
    ba_init = ship.fa_PJ.sum() * reg[s].ba_init
    ba_last = ship.fa_PJ.sum() * reg[s].ba_last

    # Calculate the annual additional changing rate (1st derivitive of resulting ba curve)
    x = (2 * (ba_last - ba_init)) / ((len(year) - 1) ** 2)
    tvdf.loc[:, 'ba_PJ'] = ba_init + x * ((tvdf.index - year[0]) ** 2) / 2

    # Write time variant dataframe to csv
    tvdf.to_csv(path + s + '/ba.csv', encoding='utf8')